In [4]:
import httpx
from bs4 import BeautifulSoup
import asyncio
import yfinance as yf
import json
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
import os

In [12]:
from playwright.async_api import async_playwright
from playwright.sync_api import sync_playwright

In [ ]:
async def get_webpage(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        html = await page.content()
        await browser.close()
        return html

# Example usage
async def fetch_pages():
    urls = [
        "https://www.example.com",
        "https://www.python.org",
        "https://www.google.com"
    ]
    results = []

    tasks = [asyncio.create_task(get_webpage(url)) for url in urls]
    results = await asyncio.gather(*tasks)
    
    # for url in urls:
    #     html = await get_webpage(url)
    #     soup = BeautifulSoup(html, "html.parser")
    #     results.append((url, soup))
    
    return results

# To run this:
results = await fetch_pages()

In [22]:
results[0]

TypeError: '_GatheringFuture' object is not subscriptable

In [19]:
results

[('https://www.example.com',
  <!DOCTYPE html>
  <html><head>
  <title>Example Domain</title>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <style type="text/css">
      body {
          background-color: #f0f0f2;
          margin: 0;
          padding: 0;
          font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
          
      }
      div {
          width: 600px;
          margin: 5em auto;
          padding: 2em;
          background-color: #fdfdff;
          border-radius: 0.5em;
          box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
      }
      a:link, a:visited {
          color: #38488f;
          text-decoration: none;
      }
      @media (max-width: 700px) {
          div {
              margin: 0 auto;
              width: auto;
          }
      }
      </style>
 

In [17]:
from playwright.sync_api import sync_playwright

urls = [
    "https://www.bloomberg.com/quote/SPX:IND",
    "https://www.bloomberg.com/quote/CCMP:IND",
]
soups = []
async with sync_playwright() as p:
    browser = p.chromium.launch(headless=True)
    page = browser.new_page()
    for i, url in urls:
        page.goto(url)
        html = page.content()
        soups.append((i, BeautifulSoup(html, "html.parser")))
    browser.close()

TypeError: 'PlaywrightContextManager' object does not support the asynchronous context manager protocol

In [6]:
async def get_pw_url(playwrightm, url):
    browser = await playwright.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto('https://www.pw.org/literary_events')
    html = await page.content()
    await browser.close()

In [10]:
urls = [
    'https://playwright.dev/python/docs/api/class-playwright',
    'https://playwright.dev/python/docs/api/class-apirequest',
]

async with async_playwright() as playwright:
    for url in urls:
        tasks = [asyncio.create_task(get_pw_url(playwright, url))]
        responses = await asyncio.gather(*tasks)

TimeoutError: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://www.pw.org/literary_events", waiting until "load"


In [9]:
responses

NameError: name 'responses' is not defined

In [13]:
import ollama

with open("/home/ibrahim/stock/logs/prompt_template copy.txt", "r") as f:
    prompt = f.read()

response = ollama.generate(
        model = "qwen2.5:3b",
        prompt=prompt,
        format="json",
    )

print(response)

model='qwen2.5:3b' created_at='2025-03-19T02:41:28.272443298Z' done=True done_reason='stop' total_duration=10976819361 load_duration=14345924 prompt_eval_count=853 prompt_eval_duration=570000000 eval_count=662 eval_duration=10391000000 response='{\n    "analysis": {\n        "news_summary": {\n            "title": "NVIDIA (NVDA) Reports Mixed Performance with New Competitors and Market Threats",\n            "date": "2023-10-04",\n            "key_points": [\n                "- NVIDIA\'s stock fell by 3.43% to $115.43 per share in the latest quarter.",\n                "- NVIDIA maintained its leadership position in the GPU market despite regulatory risks and market threats from emerging technologies like Avidity Biosciences Inc. and Lucid Group.",\n                "- Regulatory concerns include data privacy issues, antitrust litigation, and potential anticompetitive behavior within the AI sector."\n            ],\n            "sentiment": {\n                "overall": "Mixed",\n      

In [23]:
analysis = json.loads(response.response)["analysis"]

In [24]:
analysis

{'news_summary': {'title': 'NVIDIA (NVDA) Reports Mixed Performance with New Competitors and Market Threats',
  'date': '2023-10-04',
  'key_points': ["- NVIDIA's stock fell by 3.43% to $115.43 per share in the latest quarter.",
   '- NVIDIA maintained its leadership position in the GPU market despite regulatory risks and market threats from emerging technologies like Avidity Biosciences Inc. and Lucid Group.',
   '- Regulatory concerns include data privacy issues, antitrust litigation, and potential anticompetitive behavior within the AI sector.'],
  'sentiment': {'overall': 'Mixed',
   'market_reaction': "Decline in NVIDIA's stock price, with a loss of 3.43%.",
   'analyst_concerns': ['Regulatory risks (data privacy issues and antitrust litigation)',
    'Market threats from emerging technologies']}},
 'stock_impact': {'short_term': {'price_movement': '-3.43%',
   'catalysts': ["NVIDIA's performance amid new competitors and regulatory risks"],
   'risks': ['Regulatory pressures',
   

In [27]:
print(analysis.keys())

dict_keys(['news_summary', 'stock_impact', 'technical_factors', 'financial_metrics', 'recommendations'])


In [26]:
for keys in analysis.keys():
    print(analysis[keys].keys())

dict_keys(['title', 'date', 'key_points', 'sentiment'])
dict_keys(['short_term', 'long_term'])
dict_keys(['product_launches', 'AI_innovation', 'data_center_growth'])
dict_keys(['revenue_impact', 'margin_pressure', 'guidance_changes'])
dict_keys(['trading_strategy', 'hold/buy/sell', 'monitor_factors'])


In [10]:
help(ollama.generate)

Help on method generate in module ollama._client:

generate(model: str = '', prompt: Optional[str] = None, suffix: Optional[str] = None, *, system: Optional[str] = None, template: Optional[str] = None, context: Optional[Sequence[int]] = None, stream: bool = False, raw: Optional[bool] = None, format: Union[Literal['', 'json'], Dict[str, Any], NoneType] = None, images: Optional[Sequence[Union[str, bytes]]] = None, options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None, keep_alive: Union[float, str, NoneType] = None) -> Union[ollama._types.GenerateResponse, collections.abc.Iterator[ollama._types.GenerateResponse]] method of ollama._client.Client instance
    Create a response using the requested model.

    Raises `RequestError` if a model is not provided.

    Raises `ResponseError` if the request could not be fulfilled.

    Returns `GenerateResponse` if `stream` is `False`, otherwise returns a `GenerateResponse` generator.



In [184]:
import os

directory_path = '/home/ibrahim/stock/data'

# Get list of all files and directories in the specified directory
years = os.listdir(directory_path)

total_articles = 0

for year in years:
    months = os.listdir(os.path.join(directory_path, year))
    files = os.listdir(os.path.join(directory_path, year))
    for file in files:
        with open(os.path.join(directory_path, year, file), 'r') as f:
            data = json.load(f)
            total_articles += data["totalResults"]

print(total_articles)

8297


In [181]:
os.listdir("/home/ibrahim/stock/data")

['2020', '2024', '2022', '2021', '2023']

In [74]:
!pip install feedparser


In [66]:
from pygooglenews import GoogleNews 
gn = GoogleNews()
nvda_response = gn.search("NVDA", from_= "2021-01-01", to_="2021-01-31", )
nvidia_response = gn.search("Nvidia", from_= "2021-01-01", to_="2021-31-01")
nvidia_stock = gn.search("Nvidia stock", from_= "2021-01-01", to_="2021-31-01")

In [67]:
len(nvidia_stock["entries"])

48

In [70]:
nvidia_stock["entries"][5]["link"]

'https://news.google.com/rss/articles/CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU?oc=5'

In [75]:
import feedparser

url = 'https://news.google.com/rss/articles/CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU?oc=5'

feed = feedparser.parse(url)

In [90]:
response = httpx.get(url)

In [95]:
response2 = httpx.get(response.headers['location'])

In [100]:
response2.content

b'<!doctype html><html lang="en-US" dir="ltr"><head><base href="https://news.google.com/"><link rel="preconnect" href="//www.gstatic.com"><meta name="referrer" content="origin"><link rel="canonical" href="https://news.google.com/rss/articles/CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU"><meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui"><meta name="google-site-verification" content="AcBy5YFny2HQgVUCR18tO5YUTf6MpVlcJqGTd-a9-SI"><meta name="mobile-web-app-capable" content="yes"><meta name="apple-mobile-web-app-capable" content="yes"><meta name="application-name" content="News"><meta name="apple-mobile-web-app-title" content="News"><meta name="apple-mobile-web-app-status-bar-style" content="black"><meta name="theme-color" content="white"><meta name="msapplication-tap-highlight" content="no"><link rel="shor

In [80]:
response = httpx.get("https://news.google.com/rss/articles/CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU")

In [85]:
httpx.get(response.headers["location"]).content

b'<!doctype html><html lang="en-US" dir="ltr"><head><base href="https://news.google.com/"><link rel="preconnect" href="//www.gstatic.com"><meta name="referrer" content="origin"><link rel="canonical" href="https://news.google.com/rss/articles/CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU"><meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui"><meta name="google-site-verification" content="AcBy5YFny2HQgVUCR18tO5YUTf6MpVlcJqGTd-a9-SI"><meta name="mobile-web-app-capable" content="yes"><meta name="apple-mobile-web-app-capable" content="yes"><meta name="application-name" content="News"><meta name="apple-mobile-web-app-title" content="News"><meta name="apple-mobile-web-app-status-bar-style" content="black"><meta name="theme-color" content="white"><meta name="msapplication-tap-highlight" content="no"><link rel="shor

In [76]:
links = []

for entry in feed.entries:
    links.append((
        entry.title,
        entry.link
    ))

print(links)


[]


In [101]:
import base64

In [135]:
url = 'https://news.google.com/rss/articles/CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU?oc=5'
base64_url = url.replace("https://news.google.com/rss/articles/", "").split("?")[0]

print(base64_url)
# response = httpx.get(actual_url)
# if response.status_code == 302:
#     response = httpx.get(response.headers["location"])

# soup = BeautifulSoup(response.content, "html.parser")

# soup

CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU


In [142]:
!pip install pybase64

In [150]:
import base64

encoded_string = "CBMikwFBVV95cUxOVlJUbmpVcGRWTFFiRVZUdlBkcUFYVW5tRFloRnBpX3pmSWNlSmNxXzlOUFRWd21ZcGlHZkF2bnp1aU5jMFVMNlJjSm9rMlJpalBSd3hfMGZlTVdLRG9fUXRUTHp6S3RTaHF3bzgzdVV1UHZfMndiWUtQY0JDeXlZLXdKUDdkWFE4dHVTVm1la19FdEU"

# Step 1: Remove prefix
base64_string = encoded_string[6:]

# Step 2: Replace URL-safe characters
base64_string = base64_string.replace("-", "+").replace("_", "/")

# Step 3: Add padding
padding = len(base64_string) % 4
if padding != 0:
    base64_string += "=" * (4 - padding)

# Step 4: Decode
try:
    binary_data = base64.urlsafe_b64decode(base64_string)
    decoded_url = binary_data.decode("utf-8")
    print(f"Decoded URL: {decoded_url}")
except Exception as e:
    print(f"Error during decoding: {e}")

Error during decoding: Invalid base64-encoded string: number of data characters (197) cannot be 1 more than a multiple of 4


In [179]:
import json
from urllib.parse import quote, urlparse

import requests
from bs4 import BeautifulSoup


def get_decoding_params(gn_art_id):
    response = requests.get(f"https://news.google.com/articles/{gn_art_id}")
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "lxml")
    div = soup.select_one("c-wiz > div")
    return {
        "signature": div.get("data-n-a-sg"),
        "timestamp": div.get("data-n-a-ts"),
        "gn_art_id": gn_art_id,
    }


def decode_urls(articles):
    articles_reqs = [
        [
            "Fbv4je",
            f'["garturlreq",[["X","X",["X","X"],null,null,1,1,"US:en",null,1,null,null,null,null,null,0,1],"X","X",1,[1,1,1],1,1,null,0,0,null,0],"{art["gn_art_id"]}",{art["timestamp"]},"{art["signature"]}"]',
        ]
        for art in articles
    ]
    payload = f"f.req={quote(json.dumps([articles_reqs]))}"
    headers = {"content-type": "application/x-www-form-urlencoded;charset=UTF-8"}
    response = requests.post(
        url="https://news.google.com/_/DotsSplashUi/data/batchexecute",
        headers=headers,
        data=payload,
    )
    response.raise_for_status()
    return [json.loads(res[2])[1] for res in json.loads(response.text.split("\n\n")[1])[:-2]]

# Example usage
encoded_urls = [
    "https://news.google.com/rss/articles/CBMipgFBVV95cUxPWV9fTEI4cjh1RndwanpzNVliMUh6czg2X1RjeEN0YUctUmlZb0FyeV9oT3RWM1JrMGRodGtqTk1zV3pkNEpmdGNxc2lfd0c4LVpGVENvUDFMOEJqc0FCVVExSlRrQmI3TWZ2NUc4dy1EVXF4YnBLaGZ4cTFMQXFFM2JpanhDR3hoRmthUjVjdm1najZsaFh4a3lBbDladDZtVS1FMHFn?oc=5",
    "https://news.google.com/rss/articles/CBMi3AFBVV95cUxOX01TWDZZN2J5LWlmU3hudGZaRDh6a1dxUHMtalBEY1c0TlJSNlpieWxaUkxUU19MVTN3Y1BqaUZael83d1ctNXhaQUtPM0IyMFc4R3VydEtoMmFYMWpMU1Rtc3BjYmY4d3gxZHlMZG5NX0s1RmR2ZXI5YllvdzNSd2xkOFNCUTZTaEp3b0IxZEJZdVFLUDBNMC1wNGgwMGhjRG9HRFpRZU5BMFVIYjZCOWdWcHI1YzdoVHFWYnZSOEFwQ0NubGx3Rzd0SHN6OENKMXZUcHUxazA5WTIw?hl=en-US&gl=US&ceid=US%3Aen",
]
articles_params = [get_decoding_params(urlparse(url).path.split("/")[-1]) for url in encoded_urls]
decoded_urls = decode_urls(articles_params)
print(decoded_urls)

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://news.google.com/articles/CBMipgFBVV95cUxPWV9fTEI4cjh1RndwanpzNVliMUh6czg2X1RjeEN0YUctUmlZb0FyeV9oT3RWM1JrMGRodGtqTk1zV3pkNEpmdGNxc2lfd0c4LVpGVENvUDFMOEJqc0FCVVExSlRrQmI3TWZ2NUc4dy1EVXF4YnBLaGZ4cTFMQXFFM2JpanhDR3hoRmthUjVjdm1najZsaFh4a3lBbDladDZtVS1FMHFn&q=EgSQ1v9lGNHr5r4GIjBUu0RGWBO4lIDKz74DR2PuU3tbIvkMT8dOdbAgJJsXgpzKAs4pNuaNsZQJXvwd86syAXJaAUM

In [176]:
help(httpx.get)

Help on function get in module httpx:

get(url: 'URL | str', *, params: 'QueryParamTypes | None' = None, headers: 'HeaderTypes | None' = None, cookies: 'CookieTypes | None' = None, auth: 'AuthTypes | None' = None, proxy: 'ProxyTypes | None' = None, follow_redirects: 'bool' = False, verify: 'ssl.SSLContext | str | bool' = True, timeout: 'TimeoutTypes' = Timeout(timeout=5.0), trust_env: 'bool' = True) -> 'Response'
    Sends a `GET` request.

    **Parameters**: See `httpx.request`.

    Note that the `data`, `files`, `json` and `content` parameters are not available
    on this function, as `GET` requests should not include a request body.



In [ ]:
import base64

google_url = "https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LnB1c2hzcXVhcmUuY29tL2d1aWRlcy9ob3Jpem9uLWZvcmJpZGRlbi13ZXN0LWJ1cm5pbmctc2hvcmVzLWFsbC1kZWx2ZXJzLXRyaW5rZXRzLWxvY2F0aW9uc9IBAA?hl=en-CA&gl=CA&ceid=CA%3Aen"
base64_url = google_url.replace("https://news.google.com/articles/","").split("?")[0]
actual_url = base64.b64decode(base64_url).decode('utf-8')

Error: Incorrect padding

In [ ]:
len()

NameError: name 'feedparser' is not defined

In [2]:
response = httpx.get("https://www.marketwatch.com/story/nvidias-stock-sinks-wiping-out-six-months-of-gains-as-the-ai-trade-unravels-cb206437")

In [5]:
response.url

URL('https://www.marketwatch.com/story/nvidias-stock-sinks-wiping-out-six-months-of-gains-as-the-ai-trade-unravels-cb206437')

In [ ]:
with open("../keys.json", "r") as f:
    keys = json.load(f)

dict_keys(['ALPHA_VANTAGE_API_KEY'])

In [2]:
with open("/home/ibrahim/stock/data/news_feed1.json", "r") as file:
    data = json.load(file)

In [15]:
async with httpx.AsyncClient() as client:
    response = await client.get(data["feed"][0]["url"])

response.content

b'<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>Trump Trade Policy A &#x27;Big Headache&#x27; For Customers, Says Apple-Supplier Foxconn&#x27;s CEO: &#x27;Very, Very Hard To Predict How Things Will Develop...&#x27; - Apple (NASDAQ:AAPL), Amazon.com (NASDAQ:AMZN) - Benzinga</title><script type="application/ld+json">{ "@context": "http://schema.org", "@type": "NewsArticle", "publisher": {"@type":"Organization","logo":{"@type":"ImageObject","url":"/next-assets/images/schema-publisher-logo-benzinga.png"},"name":"Benzinga","url":"https://www.benzinga.com"}, "mainEntityOfPage": { "@type": "WebPage", "@id": "https://www.benzinga.com/media/25/03/44349466/trump-trade-policy-a-big-headache-for-customers-says-apple-supplier-foxconns-ceo-very-very-hard-to-predict-how-things-will-develop" }, "mainEntity": "WebPage", "headline": "Trump Trade Policy A \'Big Headache\' For Customers, Says Apple-Supplier Foxconn\'s CEO: \'Very, Ve

In [13]:
url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=NVDA&apikey={keys["ALPHA_VANTAGE_API_KEY"]}"

In [14]:
response = httpx.get(url)

In [17]:
data = response.json()

In [32]:
async def get_response(client: httpx.AsyncClient, url: str, idx: int):
    response = await client.get(url)

    return (idx, response)

In [40]:
async with httpx.AsyncClient() as client:
    tasks = [get_response(client, feed["url"], idx) for idx, feed in enumerate(data["feed"])]
    responses = await asyncio.gather(*tasks)

In [43]:
for i, response in responses:
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        data["feed"][i]["text"] = soup.get_text()
    
    else:
        data["feed"][i]["text"] = None

In [46]:
with open("../data/news_feed1.json", "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
for i, feed in enumerate(data["feed"]):
    news_response = httpx.get(feed["url"])
    soup = BeautifulSoup(news_response.text, "html.parser")

    data["feed"][i]["html_response"] = str(soup)
    

<Response [200 OK]>

In [17]:
options = Options()
options.add_argument("--headless")

driver = Chrome(options=options)
driver.get(data["feed"][2]["url"])

soup = BeautifulSoup(driver.page_source, "html.parser")
print(soup.text)



























































Can This Unstoppable Stock Join Apple, Microsoft, Nvidia, Amazon, Alphabet, and Meta Platforms in the $1 Trillion Club by 2035? | The Motley Fool










Please ensure Javascript is enabled for purposes of website accessibility



























Skip to main contentEnable accessibility for low visionOpen the accessibility menu













Search







▲ S&P 500
+164%
|
▲ Stock Advisor
+830%
Join The Motley Fool






Accessibility

          Log In
        

        Help
      




Accessibility Menu

































          Our Services
          
angle-down








    Our Purpose:
  

    To make the world smarter, happier, and richer.
  
Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer. The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Foo

In [47]:
news_response.text

'<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>Trump Trade Policy A &#x27;Big Headache&#x27; For Customers, Says Apple-Supplier Foxconn&#x27;s CEO: &#x27;Very, Very Hard To Predict How Things Will Develop...&#x27; - Apple (NASDAQ:AAPL), Amazon.com (NASDAQ:AMZN) - Benzinga</title><script type="application/ld+json">{ "@context": "http://schema.org", "@type": "NewsArticle", "publisher": {"@type":"Organization","logo":{"@type":"ImageObject","url":"/next-assets/images/schema-publisher-logo-benzinga.png"},"name":"Benzinga","url":"https://www.benzinga.com"}, "mainEntityOfPage": { "@type": "WebPage", "@id": "https://www.benzinga.com/media/25/03/44349466/trump-trade-policy-a-big-headache-for-customers-says-apple-supplier-foxconns-ceo-very-very-hard-to-predict-how-things-will-develop" }, "mainEntity": "WebPage", "headline": "Trump Trade Policy A \'Big Headache\' For Customers, Says Apple-Supplier Foxconn\'s CEO: \'Very, Ver

In [31]:
BeautifulSoup(news_response.text, "html.parser")

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>Trump Trade Policy A 'Big Headache' For Customers, Says Apple-Supplier Foxconn's CEO: 'Very, Very Hard To Predict How Things Will Develop...' - Apple (NASDAQ:AAPL), Amazon.com (NASDAQ:AMZN) - Benzinga</title><script type="application/ld+json">{ "@context": "http://schema.org", "@type": "NewsArticle", "publisher": {"@type":"Organization","logo":{"@type":"ImageObject","url":"/next-assets/images/schema-publisher-logo-benzinga.png"},"name":"Benzinga","url":"https://www.benzinga.com"}, "mainEntityOfPage": { "@type": "WebPage", "@id": "https://www.benzinga.com/media/25/03/44349466/trump-trade-policy-a-big-headache-for-customers-says-apple-supplier-foxconns-ceo-very-very-hard-to-predict-how-things-will-develop" }, "mainEntity": "WebPage", "headline": "Trump Trade Policy A 'Big Headache' For Customers, Says Apple-Supplier Foxconn's CEO: 'Very, Very Hard To", "url": "https://w

In [13]:
urls = [
    "https://www.example.com",
    "https://www.oreilly.com",
    "https://www.python.org",
    "https://www.github.com",
    "https://www.stackoverflow.com",
    "https://www.reddit.com",
    "https://www.medium.com",
    "https://www.wikipedia.org",
    "https://www.linkedin.com",
    "https://www.twitter.com"
]

async with httpx.AsyncClient() as client:
    tasks = [asyncio.create_task(get_response(client, url)) for url in urls]
    responses = await asyncio.gather(*tasks)

In [16]:
with httpx.Client() as client:
    responses = [client.get(url) for url in urls]

In [21]:
responses[1].url

URL('https://www.oreilly.com')